In [12]:
# test_api.py
import requests
import pytest

# The base URL of the API
BASE_URL = "http://127.0.0.1:5000"

# A sample prompt to test
PROMPT = "draw me a cat"

def test_generate_image():
    # Send a POST request to the /generate endpoint with the prompt as JSON data
    response = requests.post(BASE_URL + "/generate", json={"prompt": PROMPT})

    # Check that the status code is 200 (OK)
    assert response.status_code == 200

    # Check that the response is a valid JSON
    assert response.headers["Content-Type"] == "application/json"

    # Check that the response contains an imageUrl key
    assert "imageUrl" in response.json()

    # Check that the imageUrl value is a valid URL
    assert response.json()["imageUrl"].startswith("http")

test_generate_image()


In [2]:
from flask import Flask, request, jsonify
from collections import defaultdict
import uuid

app = Flask(__name__)

# In-memory storage for token keys
# Replace this with a persistent storage solution (e.g., database) in a production environment
user_tokens = defaultdict(list)

@app.route('/api/keys/purchase', methods=['POST'])
def purchase_keys():
    address = request.json.get('address')
    if not address:
        return jsonify({'error': 'Invalid address'}), 400

    # Generate 10 new token keys for the user
    for _ in range(10):
        user_tokens[address].append(str(uuid.uuid4()))

    return jsonify({'balance': len(user_tokens[address])})

@app.route('/api/keys/balance', methods=['GET'])
def get_balance():
    address = request.args.get('address')
    if not address:
        return jsonify({'error': 'Invalid address'}), 400

    return jsonify({'balance': len(user_tokens[address])})

@app.route('/api/keys/consume', methods=['POST'])
def consume_key():
    address = request.json.get('address')
    if not address:
        return jsonify({'error': 'Invalid address'}), 400

    if not user_tokens[address]:
        return jsonify({'error': 'No token keys available'}), 400

    # Consume a token key
    user_tokens[address].pop(0)

    return jsonify({'balance': len(user_tokens[address])})

if __name__ == '__main__':
    app.run(host='localhost', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


In [13]:
import requests

api_base_url = 'http://localhost:5000'

def test_purchase_prompts(address):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(f'{api_base_url}/api/keys/purchase', json={'address': address}, headers=headers)
    if response.status_code == 200:
        print("Purchase prompts successful.")
        return response.json()
    else:
        print("Purchase prompts failed.")
        return None

def test_fetch_token_key_balance(address):
    response = requests.get(f'{api_base_url}/api//keys/balance?address={address}')
    if response.status_code == 200:
        print("Fetch token key balance successful.")
        return response.json()
    else:
        print("Fetch token key balance failed.")
        return None

def test_consume_token_key(address):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(f'{api_base_url}/api/keys/consume', json={'address': address}, headers=headers)
    if response.status_code == 200:
        print("Consume token key successful.")
        return response.json()
    else:
        print("Consume token key failed.")
        return None

if __name__ == "__main__":
    test_address = '0x742d35Cc6634C0532925a3b844Bc454e4438f44e'

    print("Testing purchase prompts:")
    purchase_result = test_purchase_prompts(test_address)
    print(purchase_result)

    print("\nTesting fetch token key balance:")
    balance_result = test_fetch_token_key_balance(test_address)
    print(balance_result)

    print("\nTesting consume token key:")
    consume_result = test_consume_token_key(test_address)
    print(consume_result)



Testing purchase prompts:
Purchase prompts successful.
{'balance': 20}

Testing fetch token key balance:
Fetch token key balance successful.
{'balance': 20}

Testing consume token key:
Consume token key successful.
{'balance': 19}
